In [ ]:
import os
import sys

PATH_FLOD = os.path.join(os.getcwd(), '..')

sys.path.append(os.path.abspath(PATH_FLOD))

In [ ]:
from experiments.experiments import get_dataset_from_path, get_datasets


X, y = get_dataset_from_path(get_datasets()['pen-global'])

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

counter = Counter(y)

plt.pie(counter.values(), labels=counter.keys(), autopct=lambda pct: f"{pct:.1f}% ({int(pct * sum(counter.values()) / 100)})")
plt.show()

In [ ]:
from flod.classifiers.bsvclassifier import BSVClassifier

clf = BSVClassifier(c=1.0, q=1.0, normal_class_label=1, outlier_class_label=-1)
clf.fit(X,y)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

ConfusionMatrixDisplay.from_predictions(y, clf.predict(X))
plt.show()

In [ ]:
plt.hist(clf.betas_.detach().numpy(), color='blue', alpha=0.5, label='Client 0')
plt.xlabel('Beta Values')
plt.ylabel('Frequency')
plt.title('Histogram of Beta Values')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y, clf.predict(X))

In [ ]:
confusion_matrix(y, clf.predict(X))/len(y)